## Install R Packages 

In [ ]:
# if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
# BiocManager::install(c("harmony"))
# BiocManager::install(c("glmGamPoi"))
# BiocManager::install('limma')
# install.packages("Seurat")
# install.packages("sctransform")
# install.packages('hdf5r')
# install.packages('devtools')
# devtools::install_github('immunogenomics/presto')

## Import R Packages 

In [ ]:
# Library import

library(Seurat)
library(ggplot2)
library(sctransform)
library(harmony)
library(patchwork)
library(dplyr)
library(magrittr)

library(future)
plan("multicore", workers = 10)
options(future.globals.maxSize = 12000 * 1024^2)
future.seed=TRUE 

set.seed(1)

## Import Visium Data

In [ ]:
data_dir <- './'
list.files(data_dir) # Should show filtered_feature_bc_matrix.h5
Visium_1 <- Load10X_Spatial(data.dir = data_dir)

## QC Visium Data

In [ ]:
plot1 <- VlnPlot(Visium, features = "nCount_Spatial", pt.size = 0.1) + NoLegend()
plot2 <- SpatialFeaturePlot(Visium, features = "nCount_Spatial") + theme(legend.position = "right")
plot1
plot2

In [ ]:
Visium <- subset(Visium,  nCount_Spatial > 0)
SpatialFeaturePlot(Visium, features = "nCount_Spatial") + theme(legend.position = "right")

## Data Normalization

#### SCTransform 

In [ ]:
Visium <- SCTransform(Visium, assay = "Spatial", verbose = TRUE)
Visium

## STdeconvolve 

In [ ]:
# if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
# BiocManager::install(c("STdeconvolve"))

In [ ]:
library(parallel)
library(STdeconvolve)

In [ ]:
DefaultAssay(Visium) <- 'SCT'
counts <- as.matrix(GetAssayData(object = Visium, assay = 'Spatial', slot = "counts"), sparse = TRUE)
pos <- GetTissueCoordinates(Visium)
colnames(pos) <- c("x", "y")

#Choose 1000 top variable genes for further cell-type deconvolution
corpus <- restrictCorpus(counts, removeAbove=1.0, removeBelow = 0.05)
head(corpus)

In [ ]:
#corpus <- counts[which(rownames(counts)%in%immune_genes),]

#Fit LDA models to the data
ldas <- fitLDA(t(as.matrix(corpus)),ncores = detectCores(),seq(4,12,1))
ldas

In [ ]:
optLDA <- optimalModel(models = ldas, opt = 8)
results <- getBetaTheta(optLDA, perc.filt = 0.05, betaScale = 1000)
deconProp <- results$theta
deconGexp <- results$beta

In [ ]:
ps <- lapply(colnames(deconProp), function(celltype) {
  
  celltype <- as.numeric(celltype)
  ## highly expressed in cell-type of interest
  highgexp <- names(which(deconGexp[celltype,] > 3))
  ## high log2(fold-change) compared to other deconvolved cell-types
  log2fc <- sort(log2(deconGexp[celltype,highgexp]/colMeans(deconGexp[-celltype,highgexp])), decreasing=TRUE)
  markers <- names(log2fc)[1] ## label just the top gene
  markers
  
  # -----------------------------------------------------
  ## visualize the transcriptional profile
  dat <- data.frame(values = as.vector(log2fc), genes = names(log2fc), order = seq(length(log2fc)))
  # Hide all of the text labels.
  dat$selectedLabels <- ""
  dat$selectedLabels[1] <- markers
  
  plt <- ggplot2::ggplot(data = dat) +
    ggplot2::geom_col(ggplot2::aes(x = order, y = values,
                                   fill = factor(selectedLabels == ""),
                                   color = factor(selectedLabels == "")), width = 1) +
    
    ggplot2::scale_fill_manual(values = c("darkblue",
                                          "darkblue"
                                          )) +
    ggplot2::scale_color_manual(values = c("darkblue",
                                          "darkblue"
                                          )) +
    
    ggplot2::scale_y_continuous(expand = c(0, 0), limits = c(min(log2fc) - 0.3, max(log2fc) + 0.3)) +
    # ggplot2::scale_x_continuous(expand = c(0, 0), limits = c(-2, NA)) +
    
    ggplot2::labs(title = paste0("X", celltype),
                  x = "Gene expression rank",
                  y = "log2(FC)") +
    
    ## placement of gene symbol labels of top genes
    ggplot2::geom_text(ggplot2::aes(x = order+1, y = values-0.1, label = selectedLabels), color = "red") +
    
    ggplot2::theme_classic() +
    ggplot2::theme(axis.text.x = ggplot2::element_text(size=15, color = "black"),
                   axis.text.y = ggplot2::element_text(size=15, color = "black"),
                   axis.title.y = ggplot2::element_text(size=15, color = "black"),
                   axis.title.x = ggplot2::element_text(size=15, color = "black"),
                   axis.ticks.x = ggplot2::element_blank(),
                   plot.title = ggplot2::element_text(size=15),
                   legend.text = ggplot2::element_text(size = 15, colour = "black"),
                   legend.title = ggplot2::element_text(size = 15, colour = "black", angle = 90),
                   panel.background = ggplot2::element_blank(),
                   plot.background = ggplot2::element_blank(),
                   panel.grid.major.y = ggplot2::element_line(size = 0.3, colour = "gray80"),
                   axis.line = ggplot2::element_line(size = 1, colour = "black"),
                   legend.position="none"
                   )
  plt
})
gridExtra::grid.arrange(
  grobs = ps,
  layout_matrix = rbind(c(1, 2, 3, 4),
                        c(5, 6, 7, 8),
                        c(9, 10, 11, 12),
                        c(13, 14, 15, 16))
)

In [ ]:
highly_expressed_genes <- lapply(colnames(deconProp), function(celltype) {
  celltype <- as.numeric(celltype)
  ## highly expressed in cell-type of interest
  highgexp <- names(which(deconGexp[celltype,] > 3))
  ## high log2(fold-change) compared to other deconvolved cell-types
  log2fc <- sort(log2(deconGexp[celltype,highgexp]/colMeans(deconGexp[-celltype,highgexp])), decreasing=TRUE)
  
  ## return a list with cell type and highly expressed genes
  list(cell_type = celltype, highly_expressed_genes = highgexp)
})

highly_expressed_genes

In [ ]:
data <- do.call(rbind, lapply(highly_expressed_genes, function(x) {
  data.frame(cell_type = x$cell_type, highly_expressed_genes = x$highly_expressed_genes)
}))
head(data)

## EnrichR Topic Annotation

In [ ]:
install.packages('devtools')
library(devtools)
install_github("wjawaid/enrichR")
install.packages('data.table')

In [ ]:
library(data.table)
library(enrichR)

In [ ]:
# enrichR https://github.com/wjawaid/enrichR
listEnrichrSites()
setEnrichrSite("Enrichr")
dbs <- listEnrichrDbs()
dbs

In [ ]:
# Select DB
dbs <- c("GO_Biological_Process_2023")

In [ ]:
# Topic 1
enriched_1 <- enrichr(c(setDT(data)[cell_type == 1, highly_expressed_genes]), dbs) 
enriched_1

In [ ]:
# Topic 2
enriched_2 <- enrichr(c(setDT(data)[cell_type == 2, highly_expressed_genes]), dbs) 
enriched_2

In [ ]:
# Topic 3
enriched_3 <- enrichr(c(setDT(data)[cell_type == 3, highly_expressed_genes]), dbs) 
enriched_3

In [ ]:
# Topic 4
enriched_4 <- enrichr(c(setDT(data)[cell_type == 4, highly_expressed_genes]), dbs) 
enriched_4

In [ ]:
# Topic 5
enriched_5 <- enrichr(c(setDT(data)[cell_type == 5, highly_expressed_genes]), dbs)
enriched_5

In [ ]:
# Topic 6
enriched_6 <- enrichr(c(setDT(data)[cell_type == 6, highly_expressed_genes]), dbs)
enriched_6

In [ ]:
# Topic 7
enriched_7 <- enrichr(c(setDT(data)[cell_type == 7, highly_expressed_genes]), dbs)
enriched_7

In [ ]:
# Topic 8
enriched_8 <- enrichr(c(setDT(data)[cell_type == 8, highly_expressed_genes]), dbs)
enriched_8

In [ ]:
DefaultAssay(Visium) <- 'SCT'
counts <- GetAssayData(object = Visium, slot = "counts")
cell_rankings <- AUCell_buildRankings(counts)

In [ ]:
# Immune Signatures

Macrophages <- c('AIF1', 'C1QA', 'C1QB', 'C1QC', 'C3AR1', 'CD163', 'CD33', 'DOK2', 'F13A1', 'FGD2', 'FOLR2', 'GPR34', 
                 'LILRB4', 'LYVE1', 'MARCO', 'MPEG1', 'MRC1', 'MS4A4A', 'MS4A4E', 'MS4A6A', 'RNASE6', 'SIGLEC1', 'SYK',
                 'TYROBP', 'VSIG4')

Neutrophils <- c('AZU1','BPI','DEFA3','DEFA4','ELANE')

B_cells <- c('CD19','CD20','IGKC','IGLC1','IGLC2','IGLC3','IGLC4','IGHG1','IGHG2','IGHG3','IGHG4','IGHM','IGHE','JCHAIN',
             'IGHA1','IGHA2')

CD4_T_cells <- c('CD4')

T_reg <- c('FOXP3','IL2RA')

CD8_T_cells <- c('CD8A','CD8B')

NK_cells <- c('NCAM1')

Cytotoxic_cells <- c('GZMA','GZMB','GZMK','GZMH','PRF1','GNLY','NKG7','IFNG','TNF')

Classic_Monocytes <- c('CD14','S100A8', 'CCL8', 'S100A9')

# Non_Classic_Monocytes <- c('L1TD1', 'LYPD2', 'CKB', 'CTTNBP2', 'VMO1', 'PAQR4', 'PCDH12', 'THAP10', 'MRAS', 'GSTA4', 
                           # 'ICAM4', 'PPM1N')

DCs <- c('FLT3')

AUC_data <- rbind(
                  # AUC_Neutrophils,
                  AUC_B_cells, 
                  AUC_CD4_T_cells, AUC_T_reg, 
                  AUC_CD8_T_cells, AUC_NK_cells, 
                  AUC_Cytotoxic_cells,
                  AUC_Classic_Monocytes,
                  # AUC_Non_Classic_Monocytes,
                  AUC_DCs)

rownames(AUC_data) <- c(
                        # 'Neutrophils',
                        'B-cells',
                        'CD4-T-cells','T-reg',
                        'CD8-T-cells','NK-cells',
                        'Cytotoxic-cells',
                        'Classic-Monocytes',
                        # 'Non_Classic_Monocytes',
                        'DCs')

AUC_data

In [ ]:
AUC_Topic_1 <- AUCell_calcAUC((setDT(data)[cell_type == 1, highly_expressed_genes]), cell_rankings)
AUC_Topic_1 <- getAUC(AUC_Topic_1)

AUC_Topic_2 <- AUCell_calcAUC((setDT(data)[cell_type == 2, highly_expressed_genes]), cell_rankings)
AUC_Topic_2 <- getAUC(AUC_Topic_2)

AUC_Topic_3 <- AUCell_calcAUC((setDT(data)[cell_type == 3, highly_expressed_genes]), cell_rankings)
AUC_Topic_3 <- getAUC(AUC_Topic_3)

AUC_Topic_4 <- AUCell_calcAUC((setDT(data)[cell_type == 4, highly_expressed_genes]), cell_rankings)
AUC_Topic_4 <- getAUC(AUC_Topic_4)

AUC_Topic_5 <- AUCell_calcAUC((setDT(data)[cell_type == 5, highly_expressed_genes]), cell_rankings)
AUC_Topic_5 <- getAUC(AUC_Topic_5)

AUC_Topic_6 <- AUCell_calcAUC((setDT(data)[cell_type == 6, highly_expressed_genes]), cell_rankings)
AUC_Topic_6 <- getAUC(AUC_Topic_6)

AUC_Topic_7 <- AUCell_calcAUC((setDT(data)[cell_type == 7, highly_expressed_genes]), cell_rankings)
AUC_Topic_7 <- getAUC(AUC_Topic_7)

AUC_Topic_8 <- AUCell_calcAUC((setDT(data)[cell_type == 8, highly_expressed_genes]), cell_rankings)
AUC_Topic_8 <- getAUC(AUC_Topic_8)

AUC_Macrophages <- AUCell_calcAUC(Macrophages, cell_rankings)
AUC_Macrophages <- getAUC(AUC_Macrophages)

# AUC_Neutrophils <- AUCell_calcAUC(Neutrophils, cell_rankings)
# AUC_Neutrophils <- getAUC(AUC_Neutrophils)

AUC_B_cells <- AUCell_calcAUC(B_cells, cell_rankings)
AUC_B_cells <- getAUC(AUC_B_cells)

AUC_CD4_T_cells <- AUCell_calcAUC(CD4_T_cells, cell_rankings)
AUC_CD4_T_cells <- getAUC(AUC_CD4_T_cells)

AUC_T_reg <- AUCell_calcAUC(T_reg, cell_rankings)
AUC_T_reg <- getAUC(AUC_T_reg)

AUC_CD8_T_cells <- AUCell_calcAUC(CD8_T_cells, cell_rankings)
AUC_CD8_T_cells <- getAUC(AUC_CD8_T_cells)

AUC_NK_cells <- AUCell_calcAUC(NK_cells, cell_rankings)
AUC_NK_cells <- getAUC(AUC_NK_cells)

AUC_Cytotoxic_cells <- AUCell_calcAUC(Cytotoxic_cells, cell_rankings)
AUC_Cytotoxic_cells <- getAUC(AUC_Cytotoxic_cells)

AUC_Classic_Monocytes <- AUCell_calcAUC(Classic_Monocytes, cell_rankings)
AUC_Classic_Monocytes <- getAUC(AUC_Classic_Monocytes)

# AUC_Non_Classic_Monocytes <- AUCell_calcAUC(AUC_Non_Classic_Monocytes, cell_rankings)
# AUC_Non_Classic_Monocytes <- getAUC(AUC_Non_Classic_Monocytes)

AUC_DCs <- AUCell_calcAUC(DCs, cell_rankings)
AUC_DCs <- getAUC(AUC_DCs)

In [ ]:
AUC_data <- rbind(AUC_Topic_1, AUC_Topic_2,
                  AUC_Topic_3, AUC_Topic_4,
                  AUC_Topic_5, AUC_Topic_6,
                  AUC_Topic_7, AUC_Topic_8,
                  AUC_Macrophages,
                  # AUC_Neutrophils,
                  AUC_B_cells, 
                  AUC_CD4_T_cells, AUC_T_reg, 
                  AUC_CD8_T_cells, AUC_NK_cells, 
                  AUC_Cytotoxic_cells,
                  AUC_Classic_Monocytes,
                  # AUC_Non_Classic_Monocytes,
                  AUC_DCs)

rownames(AUC_data) <- c('Topic 1','Topic 2',
                        'Topic 3','Topic 4',
                        'Topic 5','Topic 6',
                        'Topic 7','Topic 8',
                        'Macrophages',
                        # 'Neutrophils',
                        'B-cells',
                        'CD4-T-cells','T-reg',
                        'CD8-T-cells','NK-cells',
                        'Cytotoxic-cells',
                        'Classic-Monocytes',
                        # 'Non_Classic_Monocytes',
                        'DCs')

AUC_data

In [ ]:
Visium[['AUC']] <- CreateAssayObject(data = AUC_data)
DefaultAssay(Visium) <- 'AUC'
Visium

## PCA, UMAP, and Clustering

In [ ]:
DefaultAssay(Visium) <- 'AUC'

VariableFeatures(Visium) <- rownames(Visium[["AUC"]])

Visium <- ScaleData(Visium)

# Calculate PCs
Visium <- RunPCA(Visium,
                        reduction.name = "aucpca",
                        reduction.key = "AUCPC_",npcs = 40)

In [ ]:
ElbowPlot(Visium, ndims = 40, reduction = "aucpca")

In [ ]:
dims = 15

Visium <- FindNeighbors(Visium, reduction = "aucpca", dims = 1:dims)
Visium <- FindClusters(Visium, verbose = TRUE)
Visium <- RunUMAP(Visium, 
                             reduction = "aucpca",
                             dims = 1:dims)

In [ ]:
Visium <- FindClusters(Visium,
                                  resolution = 0.7,
                                  verbose = FALSE)

DimPlot(Visium, reduction = "umap",
        pt.size = 1.25,label = TRUE)

ggsave(
  "AUC UMAP.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1, 
#   1 plot == 10 cm in each dimention
  width = 17,
  height = 15,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
SpatialDimPlot(Visium, label = TRUE, label.size = 4)
ggsave(
  "Spatial AUC.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1, 
#   1 plot == 10 cm in each dimention
  width = 19,
  height = 15,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
features = c('Topic 1','Topic 2',
                        'Topic 3','Topic 4',
                        'Topic 5','Topic 6',
                        'Topic 7','Topic 8',
                        'Macrophages',
                        # 'Neutrophils',
                        'B-cells',
                        'CD4-T-cells','T-reg',
                        'CD8-T-cells','NK-cells',
                        'Cytotoxic-cells',
                        'Classic-Monocytes',
                        # 'Non_Classic_Monocytes',
                        'DCs')

DotPlot(Visium, 
         col.min = 0, 
        features = features) + RotatedAxis() + coord_flip() 
# + scale_colour_viridis()

# Save the last plot as a pdf

ggsave(
  "AUC Dot.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1,
#   1 plot == 10 cm in each dimention
  width = 30 ,
  height = 17,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

## Re-order and re-name clusters 

In [ ]:
levels(Visium) <- c('1','2','9',
                               '0',
                               '3',
                               '8',
                               '7',
                               '5', 
                               '4',
                               '6'
                              )

new.cluster.ids <- c('Skeletal myocytes','Skeletal myocytes','Skeletal myocytes',
                     'Fibroblasts',
                     'Immune Infiltration',
                     'CD4+ T-cells',
                     'Tumor 1 Core',
                     'Tumor 1 Edge', 
                     'Tumor 2',
                     'Tumor 3'
                     )

names(new.cluster.ids) <- levels(Visium)
Visium <- RenameIdents(Visium, new.cluster.ids)

In [ ]:
DimPlot(Visium, reduction = "umap",
        pt.size = 1,label = FALSE)

ggsave(
  "AUC Clusters UMAP.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1, 
#   1 plot == 10 cm in each dimention
  width = 20.5,
  height = 15,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
SpatialDimPlot(Visium, label = FALSE, label.size = 4)
ggsave(
  "Spatial AUC.pdf",
  plot = last_plot(),
  device = "pdf",
#   path = NULL,
#   scale = 1, 
#   1 plot == 10 cm in each dimention
  width = 19,
  height = 15,
  units = "cm",
  dpi = 10000,
  limitsize = TRUE,
#   bg = NULL,
)

In [ ]:
saveRDS(Visium, file = "Seurat_Spatial.rds")